**Import Libraries**

In [ ]:
try:
    import transformers
except ImportError:
    !pip install transformers
    import transformers
try:
    import emoji
except ImportError:
    !pip install emoji
    import emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 101 kB 10.0 MB/s 
     |████████████████████████████████| 6.6 MB 30.5 MB/s 
     |████████████████████████████████| 596 kB 57.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 4.4 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=d1946c89271c4e8186248d09c476a47f408f27f37e318406fd6b30457d2f7060
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
local_run = False
local_path = "C:\\Greenwich\\MSc Project\\project_code\\"
store_local = False

In [ ]:
import pandas as pd
import numpy as np

import random
import string
import re
import ast

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

if local_run:
    nltk.data.path.append('/Users/algin/VOLD/nltk_data')
    nltk.download('stopwords',download_dir='/Users/algin/VOLD/nltk_data')
    nltk.download('punkt',download_dir='/Users/algin/VOLD/nltk_data')
else:
    nltk.download('stopwords')
    nltk.download('punkt')
stop_words = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import TFDistilBertForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

from sklearn.metrics import confusion_matrix,f1_score

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import confusion_matrix,f1_score

import xgboost as xgb

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Load Data**

In [ ]:
# full_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_train_data.csv?raw=true")
# test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_test_data.csv?raw=true")

In [ ]:
full_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/uniform/uniform_train_data.csv?raw=true")
test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/uniform/uniform_test_data.csv?raw=true")

In [ ]:
len(full_train_data), len(test_data)

(91269, 9851)

In [ ]:
# train_data, val_data = train_test_split(full_train_data,test_size=5000,random_state=21)

In [ ]:
split = StratifiedShuffleSplit(n_splits=2,test_size=0.1, random_state=23)
for train_index, val_index in split.split(full_train_data[['tweet_text','cleaned']],full_train_data['is_cyberbullying']):
    train_data = full_train_data.loc[train_index]
    val_data = full_train_data.loc[val_index]

In [ ]:
len(train_data), len(val_data), len(test_data)

(82142, 9127, 9851)

**Splitting into labels and features**

In [ ]:
train_data.head()

,tweet_text,is_cyberbullying,cleaned
8163,@iamlabeng peace and order at the price of HR ...,1,peac order price hr violat airport lgu achiev ...
71649,"Foo Fighters edit \n\nIt wasn't disruptive, I ...",0,foo fighter edit disrupt ask valid question ex...
29745,and make a video with screenshots exposing my ...,1,make video screenshot expos bulli bulli high s...
10622,#Feminazi's gone wild! Smh!😒,1,feminazi gone wild smh
76699,"I agree. More than one overlapping articles, p...",0,agre one overlap articl practic theme neg effe...


In [ ]:
val_data.head()

,tweet_text,is_cyberbullying,cleaned
32316,"@Rileyyz_69 stupid fuck, riley isn't allowed t...",1,stupid fuck riley allow use kind social networ...
28244,all the girls from high school who bullied me ...,1,girl high school bulli like 1d realli wan na p...
45409,"== Hey, you didnt tell me how fucking long thi...",1,hey didnt tell fuck long block for wtf long wait
774,RT @NoToFeminism: I don’t need femisnn i heard...,0,rt don ’ t need femisnn heard femist tri write...
83717,}}\n{{WikiProject University of Oxford|class=B...,0,wikiproject univers oxfordclassbimportancemid


In [ ]:
test_data.head()

,tweet_text,is_cyberbullying,cleaned
0,This video could be terrible and my weave woul...,0,video could terribl weav would still snatch
1,or so I can direct parents there around xmas t...,0,direct parent around xma time p
2,Drasko trying to use his fork to eat the bread...,0,drasko tri use fork eat breadcrumb
3,@NikkiGobel hmm okay.,0,hmm okay
4,Women have been equal socially for quite awhil...,0,women equal social quit awhil lt said author t...


In [ ]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

In [ ]:
banned_list= string.punctuation
punctuation_reg_exp = "[" + banned_list + "]"
emoji_reg_exp = get_emoji_regexp()

def clean_text(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)

    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)

    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text))
    
    text = re.sub(punctuation_reg_exp,"",text)

    text = re.sub("\s\s+" , " ", text)

    text = re.sub(emoji_reg_exp, r"", text)

    return text

In [ ]:
def split_label_and_feature(data):
    data['cleaned'] = data['tweet_text'].apply(clean_text)
    return data['cleaned'], data['is_cyberbullying']

In [ ]:
X_train, y_train = split_label_and_feature(train_data)
X_val, y_val = split_label_and_feature(val_data)
X_test, y_test = split_label_and_feature(test_data)

In [ ]:
X_train.head()

8163     peace order price hr violations airport lgu ac...
71649    foo fighters edit disruptive asking valid ques...
29745    make video screenshots exposing bullies bullie...
10622                              feminazis gone wild smh
76699    agree one overlapping articles practical theme...
Name: cleaned, dtype: object

**Model Training**

In [ ]:
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

MODEL_NAME = 'roberta-base'
model = TFRobertaForSequenceClassification.from_pretrained(MODEL_NAME)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
max_length = 128
def convert_example_to_feature(text):
  return roberta_tokenizer.encode_plus(text,
                                       add_special_tokens=True,
                                       max_length=max_length,
                                       pad_to_max_length=True,
                                       return_attention_mask=True,
  )

def map_example_to_dict(input_ids, attention_masks, label):
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label

def encode_examples(ds, limit=-1):
     # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  attention_mask_list = []
  label_list = []
  if (limit > 0):
    ds = ds.take(limit)
  for text, label in tfds.as_numpy(ds):
    bert_input = convert_example_to_feature(text.decode())
    input_ids_list.append(bert_input['input_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list,
                                             attention_mask_list,
                              label_list)).map(map_example_to_dict)

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        all_tokens.append(tokens)
    return np.array(all_tokens)

In [ ]:
max_len =840
train_input = bert_encode(X_train, roberta_tokenizer, max_len=max_len)
val_input = bert_encode(X_val, roberta_tokenizer, max_len=max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (559 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
xgb_model = xgb.XGBClassifier(n_estimators=500)
xgb_model.fit(train_input, y_train)

XGBClassifier(n_estimators=500)

In [ ]:
predictions = xgb_model.predict(val_input)

In [ ]:
confusion_matrix(y_val,predictions)

array([[3451, 1062],
       [1341, 3273]])

In [ ]:
f1_score(y_val,predictions)

0.7314783774723433

**Grid Search on Best Model**

In [ ]:
#setting grid of selected parameters for iteration
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 10, 15],
        'n_estimators': [100,300,500,800,1000],
        'learning_rate': [0.01, 0.06, 0.1, 0.5,],
        }

In [ ]:
# Initialize Grid Search Model
xgb_base = xgb.XGBClassifier()
xgb_grid_model = GridSearchCV(estimator=xgb_base, param_grid=param_grid, scoring='accuracy',
                                 verbose=10, n_jobs=-1, cv=2)

xgb_grid_model = RandomizedSearchCV(estimator=xgb_base, param_distributions=param_grid, scoring='accuracy',
                                 verbose=10, n_jobs=-1, cv=2)

# Fit Grid Search Model
xgb_grid_model.fit(train_input, y_train)
print("Best score: %0.3f" % xgb_grid_model.best_score_)
print("Best parameters set:")
best_parameters = xgb_grid_model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

best_xgb_model = xgb_grid_model.best_estimator_

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best score: 0.776
Best parameters set:
	colsample_bytree: 0.6
	gamma: 5
	learning_rate: 0.06
	max_depth: 15
	min_child_weight: 5
	n_estimators: 500
	subsample: 1.0
